# Init

In [1]:
### Standard ###
import sys, os

### Special ###
import numpy as np

### Local ###
sys.path.append( "../pddlstream/" )
from pddlstream.algorithms.meta import solve, create_parser
from pddlstream.algorithms.search import solve_from_pddl
from pddlstream.utils import read
from pddlstream.language.constants import print_solution, PDDLProblem
from pddlstream.utils import read, get_file_path, Profiler
from pddlstream.language.constants import PDDLProblem, And, Equal, print_solution

# Streams

In [ ]:
class Pose:
    """ A named object, it's pose, and the surface that supports it """
    
    def __init__( self, name, pose, surf ):
        self.name = name
        self.pose = pose
        self.surf = surf # WARNING: I do not like that this is part of the predicate!


def get_pose_stream( camera, detector, robot ):
    """ ??? Return a function that returns poses ??? """
    # NOTE: ??? This function assumes that the camera, detector, and robot were initialized ???

    blockNames  = ['redBlock', 'ylwBlock', 'bluBlock',]

    def rtn_func():
        """ ??? A function that returns poses ??? """
        pcd, rgbdImage = detector.real.get_PCD()
        depthImage, colorImage = rgbdImage.depth, rgbdImage.color
        urPose = robot.get_tcp_pose()
        blocks = detector.getBlocksFromImages( colorImage, depthImage, urPose, display = False )

        for i, blockName in enumerate( blockNames ):
            pose = blocks[i].worldFrameCoords
            # WARNING: I DO NOT KNOW WHY THIS IS A `yield` STATEMENT
            yield Pose( blockName, pose, 'table' ) # WARNING: HACK, The supporting object is hardcoded

# PDDLStream Context

In [2]:
domain_pddl  = read( get_file_path( os.path.join(os.getcwd(), '30_pdls_test_01'), 'domain.pddl' ) )
stream_pddl  = read( get_file_path( os.path.join(os.getcwd(), '30_pdls_test_01'), 'stream.pddl' ) )
constant_map = {}

In [3]:
scan_cost = 1
init      = [
    Equal(('MoveCost',), 1),
    Equal(('PickCost',), 1),
    Equal(('PlaceCost',), 1),
    Equal(('ScanCost',), scan_cost),
]

In [4]:
blockNames  = ['redBlock', 'ylwBlock', 'bluBlock',]
supportName = 'table'

init += [
    ('Observable', np.eye(4)),
]

for blockName in blockNames:
    pose = np.eye(4)
    init += [
        ('Pose'  , blockName, pose), 
        ('AtPose', blockName, pose),
        ('Scannable'   , blockName),
        ('Registerable', blockName),
        ('Graspable'   , blockName),
        ('Uncertain'   , blockName),
    ]

goal = ('and',
    ('on', 'ylwBlock', 'bluBlock',),
    ('on', 'redBlock', 'ylwBlock',),
)

problem = PDDLProblem( domain_pddl, constant_map, stream_pddl, stream_map, init, goal )